# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

## Norvig's spelling corrector

In [19]:
import requests
import os

# downloading big.txt and saving it
# big.txt is a file used in the norvig's solution

#response = requests.get("https://norvig.com/big.txt")
#if response.status_code == 200:
#    with open("big.txt", 'wb') as f:
#        f.write(response.content)

In [20]:
# norvig's spelling corrector - copied from https://norvig.com/spell-correct.html

import re
from collections import Counter

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def P_norvig(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def norvig_correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P_norvig)

In [21]:
norvig_correction("speling")

'spelling'

## Custom spelling corrector based on Norvig's + bigrams

In [22]:
from collections import Counter, defaultdict

# loading bigrams and their counts from 'bigrams.txt'
BIGRAMS = defaultdict(Counter)
with open('bigrams.txt', 'r') as file:
    for line in file:
        count, word1, word2 = line.split()
        BIGRAMS[word1][word2] = int(count)
        
def prev_bigram_probability(prev_word, candidate):
    return BIGRAMS[prev_word][candidate] / sum(BIGRAMS[prev_word].values())

def next_bigram_probability(candidate, next_word):
    return sum(BIGRAMS[word][next_word] for word, next_word_counter in BIGRAMS.items() if next_word in next_word_counter) / BIGRAMS[candidate][next_word]
        
def P_bigram(candidate, prev_word=None, next_word=None): # calculate conditional probability of a candidate word given prev_word and next_word
    if prev_word is not None and next_word is not None:
        return prev_bigram_probability(prev_word, candidate) * next_bigram_probability(candidate, next_word)
    elif prev_word is not None:
        return prev_bigram_probability(prev_word, candidate)
    elif next_word is not None:
        return next_bigram_probability(candidate, next_word)
    else:
        return WORDS[candidate] / sum(WORDS.values())
    
def prev_bigram_exists(prev_word, candidate):
    return prev_word is not None and prev_word in BIGRAMS and candidate in BIGRAMS[prev_word]

def next_bigram_exists(candidate, next_word):
    return next_word is not None and candidate in BIGRAMS and next_word in BIGRAMS[candidate]

def bigram_correction(word, prev_word=None, next_word=None): # most probable spelling correction for word given prev_word and next_word
    correction_candidates = candidates(word)
    
    filtered_candidates = [candidate for candidate in correction_candidates
                           if prev_bigram_exists(prev_word, candidate) 
                           and next_bigram_exists(candidate, next_word)]
    
    if len(filtered_candidates) > 0:
        return max(filtered_candidates, key=lambda candidate: P_bigram(candidate, prev_word, next_word))
    
    filtered_candidates = [candidate for candidate in correction_candidates
                           if prev_bigram_exists(prev_word, candidate) 
                           or next_bigram_exists(candidate, next_word)]
    
    if len(filtered_candidates) > 0:
        return max(filtered_candidates, key=lambda candidate: P_bigram(candidate, 
                                                                prev_word if prev_bigram_exists(prev_word, candidate) else None, 
                                                                next_word if next_bigram_exists(candidate, next_word) else None))
    
    return max(correction_candidates, key=P_bigram)

In [27]:
bigram_correction("dking", next_word="size")

'king'

In [30]:
bigram_correction("dking", next_word="homework")

'doing'

In [31]:
def corrector(sentence, method="bigram"):
    words = sentence.split()
    corrections = []
    for i in range(len(words)):
        if not words[i].isalpha(): # if string contains punctuation then leave it as it is - it is not a word
            corrections.append(words[i])
            continue
        if method == "norvig":
            corrections.append(norvig_correction(words[i]))
        else: #if method = "bigram"
            if i != 0:
                prev_word = words[i-1]
                if not prev_word.isalpha():
                    prev_word = None
            else:
                prev_word = None
            if i != len(words) - 1:
                next_word = words[i+1]
                if not next_word.isalpha():
                    next_word = None
            else:
                next_word = None
            corrections.append(bigram_correction(words[i], prev_word=prev_word, next_word=next_word))
    return ' '.join(corrections)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I used Norvig's spelling corrector as a basis of my solution. To obtain vocabulary of words and their frequencies, I used big.txt, as in the Norvig's solution.

I used bigrams dataset specified in the assignment description. Larger datasets are harder to obtain: the ones I found were not free to use. Also, working with them would cost more computation power.

My solution has the following implementation:

First, I generate possible correction candidates that are 1 (or 2, if there are none) edits away from the original word that are in the vocabulary from big.txt (as in the Norvig's solution).

Then, for each candidate, I check if both bigrams prev_word+candidate and candidate+next_word exist (the word is not in the very beginning or very end of the sentence) and are present in the bigrams dataset.

Then, I take candidates for which both these bigrams exist and are present in the dataset and calculate conditional probability of each candidate appearing in a given context: P(candidate|prev_word,next_word) = P(candidate|prev_word) * P (candidate|next_word) = count of bigrams (prev_word, candidate) / count of all bigrams (prev_word, smth) * count of bigrams(candidate, next_word) / count of all bigrams (smth, next_word). I return the word with the highest calculated conditional probability.

If there are no candidates for which both the bigrams exist and are present in the dataset, I take the candidates for which one of the bigrams exists and is present in the dataset, calculate conditional probability of them appearing in a context with it the same way, then rank them and return the one with the highest calculated colditional probability.

If for every candidate there are no bigrams that exist and are present in the dataset, I rank the candidates by the frequency of their usage, as in the Norvig's solution, and return the one with the highest one.

To test my solution, I used collection of texts 'brown' from nltk. I randomly took 150 sentences from there, for each of them I generated a misspelled version, and then I ran them through Norvig's corrector and the one I wrote, and for each calculated the accuracy of correction. It appeared my solution that used bigrams got almost the same results as the Norvig's one - I think adding more context via using 3grams, 4grams, ... and larger datasets could help with that - using small bigrams dataset has its limits.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [41]:
# downloading dataset containing english texts
import nltk
nltk.download('brown')
from nltk.corpus import brown
sentences = brown.sents()

In [39]:
import random
import string

test_set_size = 150
sentence_list = [' '.join(sentence).lower() for sentence in sentences]
random_sentences = random.sample(sentence_list, test_set_size)

noise = 0.15
def generate_misspelling(word):
    # if input string contains punctuation then leave it as it is - it is not a word
    if not word.isalpha():
        return word
    if random.random() > noise:
        return word
    # select one of the edit functions randomly - edit1 with 80% probability
    if random.random() < 0.8:
        edit_function = edits1
    else:
        edit_function = edits2
    # apply the selected edit function to the word and return the result
    return random.choice(list(edit_function(word)))

test_set = []
for sentence in random_sentences:
    # apply random edits to randomly chosen words in the sentence to generate spelling errors
    edited_sentence = sentence
    while edited_sentence == sentence: # to ensure each sentence contains at least one spelling error
        edited_sentence = [generate_misspelling(word) for word in sentence.split()]
        edited_sentence = ' '.join(edited_sentence)
    test_set.append([edited_sentence, sentence])

In [40]:
def test(test_set, method):
    correct = 0
    for misspelled_sentence, correct_sentence in test_set:
        corrected_sentence = corrector(misspelled_sentence, method=method)
        if corrected_sentence == correct_sentence:
            correct += 1
    accuracy = correct / len(test_set)
    print("method: " + method)
    print("accuracy: " + str(accuracy))
    return
        
test(test_set, method="bigram")
test(test_set, method="norvig")

method: bigram
accuracy: 0.32
method: norvig
accuracy: 0.32666666666666666
